# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])


def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldr(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [5]:
def foldl2(f, x0, lst):
    return foldr(lambda val, acc: lambda n: acc(f(n, val)), lambda x: x,
                 lst)(x0)

In [6]:
def foldr2(f, x0, lst):
    return foldl(lambda acc, val: lambda n: acc(f(val, n)), lambda x: x,
                 lst)(x0)

In [7]:
foldr2(lambda x, y: x + y, 0, [1, 2, 3])

6

In [8]:
foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [9]:
foldl2(f, 1, [1, 2, 3]), \
foldr2(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [10]:
from collections import Counter

In [11]:
def check_inv(a, b):
    cnt = Counter(b)
    l_b = len(b)
    for _ in a:
        if cnt[_] > 0:
            cnt[_] -= 1
        else:
            if sum(cnt.values()) < l_b:
                cnt = Counter(b)
        if sum(cnt.values()) == 0:
            return True
    return False

In [12]:
a='afdgcrodtg'
b='gfd'
check_inv(a,b)

True

In [13]:
a='afdcrodtg'
b='gfd'
check_inv(a,b)

False

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [14]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value

    def __iter__(self):
        return iter(self.get_nodes())

    def __str__(self):
        return f"Tree with vertex {self.value}, left node {self.left.get_value()} and right node {self.right.get_value()}"

    def __repr__(self):
        return f"Tree({self.value}, {repr(self.left)}, {repr(self.right)})"

    def get_nodes(self):
        if self.left is None and self.right is None:
            return [self.value]
        nodes = []
        if self.left is not None:
            nodes.extend(self.left.get_nodes())
        if self.right is not None:
            nodes.extend(self.right.get_nodes())
        return nodes

    def get_value(self):
        return str(self.value)


tree = Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

print(tree)
print(repr(tree))
print(list(tree))

Tree with vertex 0, left node 1 and right node 2
Tree(0, Tree(1, Tree(3, None, None), Tree(4, None, None)), Tree(2, None, None))
[3, 4, 2]


# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [15]:
def create_polish_notation(expr):
    notation = []
    stack = []

    operations = {'(': 0, ')': 0, '+': 1, '-': 1, '*': 2, '/': 2}
    numerals = [str(x) for x in range(10)]

    number = 0
    number_flag = False
    for element in expr:
        if element in numerals:
            number_flag = True
            number = number * 10 + int(element)

        else:
            if number_flag:
                notation.append(number)
                number_flag = False
                number = 0

            if element == '(':
                stack.append(element)
            elif element == ')':
                el = stack.pop()
                while el != '(':
                    notation.append(el)
                    el = stack.pop()

            elif element in operations.keys():
                priority_of_el = operations[element]
                if not stack:
                    priority_of_el_in_stack = 0
                else:
                    priority_of_el_in_stack = operations[stack[-1]]

                while priority_of_el_in_stack >= priority_of_el:
                    notation.append(stack.pop())
                    if not stack:
                        priority_of_el_in_stack = 0
                    else:
                        priority_of_el_in_stack = operations[stack[-1]]
                stack.append(element)

    if number_flag:
        notation.append(number)
    while stack:
        notation.append(stack.pop())
    return notation

In [16]:
def calc(expr):
    polish_notation= create_polish_notation(expr)
    stack = []
    for element in polish_notation:
        if type(element) == int:
            stack.append(element)
        else:
            el2=stack.pop()
            el1=stack.pop()
            if element == '+':
                stack.append(el1+el2)
            if element == '-':
                stack.append(el1-el2)
            if element == '*':
                stack.append(el1*el2)
            if element == '/':
                stack.append(el1/el2)
    return stack.pop()
        
    
    
calc('2 * (15 - 3 * 4) - 2') == 4

True